In [2]:
import plotly.express as px
import pandas as pd
import numpy as np
import os

In [ ]:
# In here we filter human-viral strctural homologs detected by FoldSeek, to remove less relaible and spurious matches and matches of low-complexity folds
# The features used to filter were assembled in general_flow, plus, we also add pLDDT score from AlphaFold which is added in this script.
# We here use structural homologs between human and vaccinia virus.

In [3]:

def max_aln_no_gap(seq, max_gap_p = 0.05):
    start = 0
    gap_length = 0
    pos = 0
    max_length = 0 
    max_start = 0
    count = 0

    max_gap = max_gap_p * len(seq)
    
    while(pos < len(seq)):
        res = seq[pos]
        
        if (res == "-"):
            gap_length = gap_length + 1
        else:
            gap_length = 0
            
        if(max_gap < gap_length):
            if pos - start > max_length:
                max_length = pos - start - gap_length
                max_start = start
                
#             print(seq[pos])
            while (pos + 1 < len(seq)) & (seq[pos + 1] == "-"):
                pos = pos + 1
#                 print(pos)
            
            gap_length = 0
            start = pos + 1
            
        pos = pos + 1
        
        if (pos == len(seq)) & (pos - start > max_length):
            max_length = pos - start - gap_length
            max_start = start
    
    if max_length == 0:
        max_length = len(seq) 

        
    return max_start, max_start + max_length - 1, max_length

def max_seq_no_gap(seq, max_gap = 0.05):
    start, end, length = max_aln_no_gap(seq, max_gap)
    return seq[start : end]

def all_seq_ungap_index(seq, max_gap = 0.05):
    start, end, length = max_aln_no_gap(seq, max_gap)
    count = 0
    ungap_start = 0
    ungap_end = 0
#     print(len(seq))
    for i,c in enumerate(seq):
        if c == "-":
            count = count + 1
        if i == start:
            ungap_start = start - count
        if i == end - 1:
            
            ungap_end = end - count
            
    return seq.replace("-", "") ,ungap_start, ungap_end
            
        

In [4]:
def col_to_array(col):
    arr_tmp = col.replace("[", "").replace("]", "").replace("\n", "").split(" ") ### ... need to be checked
    # print(arr_tmp)
    # if "..." in arr_tmp:
    #     print(arr_tmp)
    if "..." in arr_tmp:
        return None
    arr = [float(x) for x in arr_tmp if x != ""]
    # print(arr)
    return arr

def total_median(array):
    if array == None:
        return 0
    return np.median(array)


def median_by_pos(start, end, array):
    # print(start)
    # print(end)
    # print(np.array(array))
    start = int(start)
    end = int(end)


    if array == None:
        return 0
    try:
        pos_array = np.array(array)[start -1 : end]
    except:
        print(start, end, array)
        pos_array = np.array(array)[start -1 : end]
    return np.median(pos_array)


# read data

In [8]:
before_filter_df = pd.read_csv("data/before_filter_no_plddt.csv").drop("Unnamed: 0", axis=1)
display(before_filter_df)

,virus,virus_protein,human_protein,side,full_human_abs_mean_contact_order,human_abs_mean_contact_order,virus_abs_mean_contact_order,full_virus_abs_mean_contact_order,alnlen,cigar,...,qaln,taln,alntmscore,sum_sides,qungap,tungap,qungap_len,tungap_len,ratio_ungap,ratio
0,vaccinia_wr,P01136,H3BU77-F1,-1,4.250000,4.205882,4.074074,16.190972,42,42M,...,TTTSYIPSPGIMLVLVGIIIITCCLLSVYRFTRRTKLPIQDM,PEGPRQHHPSEVTERQLANKRIQNMQHLKKEKRRLNKRFSRP,0.7793,-1,TTTSYIPSPGIMLVLVGIIIITCCLLSVYRFTRRTKLPIQD,PEGPRQHHPSEVTERQLANKRIQNMQHLKKEKRRLNKRFSR,41,41,1.000000,2.058824
1,vaccinia_wr,P01136,Q5JQD4-F1,-1,13.352113,13.352113,17.199248,16.190972,138,4M1D17M3I1M1I8M68D6I4M1I21M1D2M,...,MSMKYLMLLFAAMIIRSFADSG---N-AIETTSPEITNATTDIPAI...,SVCR-PWPAVAIALLALLVCLGALVDTCPIKPEAP-----------...,0.6074,-1,MSMKYLMLLFAAMIIRSFADSG---N-AIETTSPEITNATTDIPAI...,GEDESLEELSHYYASLCHYLNVVTRQWWEGAD-M,137,34,4.029412,2.000000
2,vaccinia_wr,P01136,Q86Y28-F1,-1,4.000000,4.000000,4.000000,16.190972,32,24M1D4M1D1I1M,...,LVLVGIIIITCCLLSVYRFTRRTKLPIQDM-V,MAAGAVFLALSAQLLQARLMKEES-PVVS-WW,0.7624,-1,LVLVGIIIITCCLLSVYRFTRRTKLPIQDM-,MAAGAVFLALSAQLLQARLMKEES-PVVS-W,31,31,1.000000,3.589744
3,vaccinia_wr,P01136,Q9NRQ5-F1,-1,4.235955,4.125000,4.074074,16.190972,52,4M1I3M1I32M2I2M1D2I1M2I1M,...,PNTT-TSY-IPSPGIMLVLVGIIIITCCLLSVYRFTRRTKL--PIQ...,MRQLKGKPKKETSKDKKERKQAMQEARQQITTVVLPTLAVVVLLI-...,0.6814,-1,PNTT-TSY-IPSPGIMLVLVGIIIITCCLLSVYRFTRRTKL--PIQ...,MRQLKGKPKKETSKDKKERKQAMQEARQQITTVVLPTLAVVVLLI-...,51,51,1.000000,2.372881
4,vaccinia_wr,P03296,P51808-F1,-1,32.555556,32.516807,45.777778,51.960448,184,8M2I5M1D9M1D3M8D2I5M1I18M1D11M8D3I5M1D6M2I9M1D...,...,KGLQTIKL--FNNEFDCIRNDIRELFKHVTDSDSIQL--PMEDN-S...,EVGFNAEEAHNIVKE-CVDGVLGGE-DYN--------HNNINQWTA...,0.5567,-1,KGLQTIKL--FNNEFDCIRNDIRELFKHVTDSDSIQL--PMEDN-S...,EVGFNAEEAHNIVKE-CVDGVLGGE-DYN--------HNNINQWTA...,183,110,1.663636,2.853448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15533,vaccinia_wr,Q89121,Q9HBY8-F1,-1,75.593297,39.517024,68.402527,67.977589,484,7M5D4M2D8M2D1M51D17M6I8M2I22M6D16M90D10M1I12M5...,...,PEYQWMSPHRLSDTVILGDCLYFNNIMSQLDLHQNWAPSVRLLNYF...,NGNINLG-----PSAN--PNAQPTDF--D-----------------...,0.5335,-1,PEYQWMSPHRLSDTVILGDCLYFNNIMSQLDLHQNWAPSVRLLNYF...,KLYFVLDYVNGGELFFHLQRERR-----FLEPRARFYAAEVASAIG...,483,236,2.046610,1.196185
15534,vaccinia_wr,Q89121,Q9HC98-F1,-1,42.312916,42.439546,68.110557,67.977589,486,4M3D1M1D3M1D3M1D4M2D2M51D16M6I9M2I21M7D12M1D2M...,...,MSPHRLSDTVILGDCLYFNNIMSQLDLHQNWAPSVRLLNYFKNFNK...,RHPN---T-LSF-RCS-LADF--QI---------------------...,0.5890,-1,MSPHRLSDTVILGDCLYFNNIMSQLDLHQNWAPSVRLLNYFKNFNK...,NELNIVLELADAGDLSQMIKYFK-----KQKRLIPERTVWKYFVQL...,485,244,1.987705,1.402556
15535,vaccinia_wr,Q89121,Q9P1W9-F1,-1,38.861635,38.861635,68.427991,67.977589,492,6M8D4M2D1I7M2D2M51D16M6I17M5I16M3D13M1D3M89D9M...,...,SPEYQWMSPHRLSDTVILGD-CLYFNNIMSQLDLHQNWAPSVRLLN...,TPTPPP--------GGKD--REAFEAEY--RL--------------...,0.6062,-1,SPEYQWMSPHRLSDTVILGD-CLYFNNIMSQLDLHQNWAPSVRLLN...,GFMLVLERPLPAQDLFDYITEKGP-----LGEGPSRCFFGQVVAAI...,491,239,2.054393,1.411576
15536,vaccinia_wr,Q89121,Q9UIK4-F1,-1,42.771125,36.393019,67.977589,67.977589,495,13M9D3M2D7M2D2M51D16M6I13M5I17M7D12M1D2M89D10M...,...,MGVANDSSPEYQWMSPHRLSDTVILGDCLYFNNIMSQLDLHQNWAP...,MFQASMRSPNMEP---------FKQ--QKVEDFY--DI--------...,0.5294,-1,MGVANDSSPEYQWMSPHRLSDTVILGDCLYFNNIMSQLDLHQNWAP...,TDVVLILELVSGGELFDFLAQK-E----SLSEEEATSFIKQILDGV...,408,237,1.721519,1.186486


# Filters

In [9]:
# list of parametres and threshold values to filter homologs 
PROTEIN_SIZE_RATIO_FILTER = 1.5
UNGAP_SIZE_RATIO_FILTER = 2.5
protein_LENGTH = 50
UNGAP_CO_FILTER = 15
PLDDT_FILTER = 70

In [10]:
print(len(before_filter_df["virus_protein"].unique()))

184


In [11]:
before_filter_df.groupby(["virus"]).agg({"human_protein":"nunique", "virus_protein":"nunique"})

,human_protein,virus_protein
virus,,
vaccinia_wr,8087,184


In [12]:
display(before_filter_df.groupby(["virus", "sum_sides"]).agg({"virus_protein":"count"}))
display(before_filter_df.groupby(["virus", "sum_sides"]).agg({"human_protein":"nunique", "virus_protein":"nunique"}))

virus_protein
virus       sum_sides               
vaccinia_wr -1                  2973
             0                  1348
             1                 11217

human_protein  virus_protein
virus       sum_sides                              
vaccinia_wr -1                  1048            141
             0                   437             67
             1                  7068            116

In [13]:
print(before_filter_df.count()[0])
print(before_filter_df["virus_protein"].nunique())
print(before_filter_df["human_protein"].nunique())
print(before_filter_df[["human_protein", "virus_protein"]].drop_duplicates().count()[0])

15538
184
8087
14864


C:\Users\TzachiHNB15\AppData\Local\Temp\ipykernel_27700\2282078749.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(before_filter_df.count()[0])
C:\Users\TzachiHNB15\AppData\Local\Temp\ipykernel_27700\2282078749.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(before_filter_df[["human_protein", "virus_protein"]].drop_duplicates().count()[0])


## size filter- filter by 'PROTEIN_LENGTH'

In [14]:
before_filter_df_length_filter = before_filter_df[(before_filter_df["qlen"] > PROTEIN_LENGTH) & (before_filter_df["tlen"] > PROTEIN_LENGTH)]

In [15]:
before_filter_df_length_filter.groupby(["virus"]).agg({"human_protein":"nunique", "virus_protein":"nunique"})

,human_protein,virus_protein
virus,,
vaccinia_wr,7484,178


## protein size ratio- filter by 'PROTEIN_SIZE_RATIO_FILTER'

In [16]:
before_filter_df_ratio_filter = (before_filter_df_length_filter[(before_filter_df_length_filter["ratio"] < PROTEIN_SIZE_RATIO_FILTER) &
                    (before_filter_df_length_filter["ratio"] > 1 / PROTEIN_SIZE_RATIO_FILTER)])

In [17]:
before_filter_df_ratio_filter.groupby(["virus"]).agg({"human_protein":"nunique", "virus_protein":"nunique"})

,human_protein,virus_protein
virus,,
vaccinia_wr,882,100


## ungap size ratio filter- filter by 'UNGAP_SIZE_RATIO_FILTER'

In [18]:
before_filter_df_ratio_filter.columns

Index(['virus', 'virus_protein', 'human_protein', 'side',
       'full_human_abs_mean_contact_order', 'human_abs_mean_contact_order',
       'virus_abs_mean_contact_order', 'full_virus_abs_mean_contact_order',
       'alnlen', 'cigar', 'qstart', 'qend', 'tstart', 'tend', 'qlen', 'tlen',
       'bits', 'qaln', 'taln', 'alntmscore', 'sum_sides', 'qungap', 'tungap',
       'qungap_len', 'tungap_len', 'ratio_ungap', 'ratio'],
      dtype='object')

In [19]:

before_filter_df_ratio_ungap_filter = (before_filter_df_ratio_filter[(before_filter_df_ratio_filter["ratio_ungap"] < UNGAP_SIZE_RATIO_FILTER )
                    & (before_filter_df_ratio_filter["ratio_ungap"] > 1 / UNGAP_SIZE_RATIO_FILTER )])

In [20]:
#before_filter_df_ratio_ungap_filter.groupby(["virus"]).agg({"human_protein":"nunique", "virus_protein":"nunique"})

## contact order filter

In [21]:
before_filter_contact_filter = (before_filter_df_ratio_ungap_filter[(before_filter_df_ratio_ungap_filter["human_abs_mean_contact_order"] > UNGAP_CO_FILTER) &
                    (before_filter_df_ratio_ungap_filter["virus_abs_mean_contact_order"] > UNGAP_CO_FILTER)])

In [22]:
before_filter_contact_filter.groupby(["virus"]).agg({"human_protein":"nunique", "virus_protein":"nunique"})

,human_protein,virus_protein
virus,,
vaccinia_wr,819,89


## pLDDT filter- filter by 'plddt' after extracting it from the AlphaFold output

In [23]:
plddt = pd.read_csv("data/exp_vaccinia_wr/plddt_vaccinia_wr.tsv", sep='\t').drop(["Unnamed: 0"], axis=1)
display(plddt)

,protein,model,score_array
0,P01136,result_model_1,[54.36326972 70.15629905 71.97217641 77.918377...
1,P01136,result_model_2,[54.60246079 67.36214912 63.67861892 73.172946...
2,P01136,result_model_3,[58.25297803 71.95361419 67.3096918 74.909556...
3,P01136,result_model_4,[55.38933037 66.31044717 67.91954371 75.547513...
4,P01136,result_model_5,[54.6864919 70.26408366 71.13292331 75.853707...
...,...,...,...
1080,Q89121,result_model_1,[40.65799413 36.47738472 33.72437501 42.063047...
1081,Q89121,result_model_2,[33.77569453 33.00878219 28.22864528 33.139072...
1082,Q89121,result_model_3,[40.2421221 38.65830327 42.32004603 40.970229...
1083,Q89121,result_model_4,[34.94388795 37.1688696 37.27431644 40.525484...


In [24]:
plddt["virus_protein"] = plddt["protein"].apply(lambda x: x.replace(".fasta", "") if ".fasta" in x else x)
plddt["virus"] = "vaccinia_wr"
plddt.drop(["protein"],axis=1, inplace = True)
plddt.head(5)

,model,score_array,virus_protein,virus
0,result_model_1,[54.36326972 70.15629905 71.97217641 77.918377...,P01136,vaccinia_wr
1,result_model_2,[54.60246079 67.36214912 63.67861892 73.172946...,P01136,vaccinia_wr
2,result_model_3,[58.25297803 71.95361419 67.3096918 74.909556...,P01136,vaccinia_wr
3,result_model_4,[55.38933037 66.31044717 67.91954371 75.547513...,P01136,vaccinia_wr
4,result_model_5,[54.6864919 70.26408366 71.13292331 75.853707...,P01136,vaccinia_wr


In [25]:
alphafold_score = pd.merge(before_filter_contact_filter[["virus_protein", "virus", "qstart", "qend", "human_protein"]], plddt, on = ["virus_protein", "virus"])
alphafold_score["score_array"] = alphafold_score["score_array"].apply(lambda x: col_to_array(x))
alphafold_score["total_score"] = alphafold_score.apply(lambda x: total_median(x["score_array"]), axis = 1).to_list()
alphafold_score["score"] = alphafold_score.apply(lambda x: median_by_pos(x["qstart"], x["qend"], x["score_array"]), axis = 1).to_list()
maxes = alphafold_score.groupby(["virus_protein", "virus"])['total_score'].idxmax()
alphafold_score_total = alphafold_score.loc[maxes].copy()
alphafold_score_total = alphafold_score_total[["virus_protein", "virus", "total_score", "score"]].drop_duplicates(["virus_protein", "virus"])

In [26]:
before_filter_plddt_filter = pd.merge(before_filter_contact_filter, alphafold_score_total, on=["virus_protein", "virus"])

In [27]:
before_filter_plddt_filter = before_filter_plddt_filter[before_filter_plddt_filter["score"] >= PLDDT_FILTER]

## save after filter

In [28]:
filtered_df = before_filter_plddt_filter.copy() 
filtered_df.columns

Index(['virus', 'virus_protein', 'human_protein', 'side',
       'full_human_abs_mean_contact_order', 'human_abs_mean_contact_order',
       'virus_abs_mean_contact_order', 'full_virus_abs_mean_contact_order',
       'alnlen', 'cigar', 'qstart', 'qend', 'tstart', 'tend', 'qlen', 'tlen',
       'bits', 'qaln', 'taln', 'alntmscore', 'sum_sides', 'qungap', 'tungap',
       'qungap_len', 'tungap_len', 'ratio_ungap', 'ratio', 'total_score',
       'score'],
      dtype='object')

In [29]:
filtered_df.to_csv("data/filtered_matches_protein_len_{}_ratio_{}_csu_{}_plddt_{}.csv"
                   .format(protein_LENGTH, PROTEIN_SIZE_RATIO_FILTER, UNGAP_CO_FILTER, PLDDT_FILTER))

In [207]:
filtered_df.groupby(["virus"]).agg({"human_protein":"nunique", "virus_protein":"nunique"})

,human_protein,virus_protein
virus,,
vaccinia_wr,809,85
